In [ ]:
import os
import numpy as np
import cv2 as cv
import gc
from tqdm.notebook import tqdm

In [ ]:
# Keras/TensorFlow Imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import RandomFlip, RandomRotation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [ ]:
# -------------------------------------------------------------------
# 1. SETUP VARIABLES AND DATA PATHS
# -------------------------------------------------------------------
IMG_SIZE = (80, 80)
channels = 1  # 1 for grayscale, 3 for color (RGB)
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
# Creating a character dictionary and sorting it
char_dict = {}
for char in os.listdir(char_path):
    # Only process directories
    if os.path.isdir(os.path.join(char_path, char)):
        try:
            char_dict[char] = len(os.listdir(os.path.join(char_path, char)))
        except Exception:
            # Handle permissions or other errors if necessary
            pass

# Sort in descending order (Standard Python Replacement for caer.sort_dict)
sorted_char_dict_list = sorted(char_dict.items(), key=lambda item: item[1], reverse=True)

In [ ]:
# Select the top 10 characters to use as classes (common for this dataset)
characters = [item[0] for item in sorted_char_dict_list[:10]]
num_classes = len(characters)

In [ ]:
print(f"Number of classes selected: {num_classes}")
print(f"Classes: {characters}")
print("-" * 30)

In [ ]:
# -------------------------------------------------------------------
# 2. CUSTOM PREPROCESSING FUNCTION (Replacement for caer.preprocess_from_dir)
# -------------------------------------------------------------------

def preprocess_images_custom(DIR, classes, IMG_SIZE, channels):
    data = []
    
    # Create a mapping from class name to numerical index (0 to num_classes-1)
    class_to_index = {class_name: i for i, class_name in enumerate(classes)}
    
    print("[INFO] Starting preprocessing...")

    for class_name in tqdm(classes, desc="Processing Classes"):
        class_path = os.path.join(DIR, class_name)
        class_index = class_to_index[class_name]
        
        if not os.path.exists(class_path):
            continue
            
        # Iterate over all files in the class directory
        for img_name in os.listdir(class_path):
            try:
                img_path = os.path.join(class_path, img_name)
                
                # Load image using OpenCV
                # Use cv.IMREAD_GRAYSCALE for channels=1, or cv.IMREAD_COLOR for channels=3
                if channels == 1:
                    img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                else:
                    # By default, load color and convert to RGB (OpenCV loads BGR)
                    img = cv.imread(img_path)
                    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
                
                if img is None:
                    continue
                    
                # Resize image
                resized_img = cv.resize(img, IMG_SIZE)
                
                # Append the image (features) and the class index (label)
                data.append([resized_img, class_index])
                
            except Exception as e:
                # print(f"Error loading {img_name}: {e}")
                pass
    
    print("[INFO] Preprocessing complete. Shuffling data...")
    # Shuffle the data
    np.random.shuffle(data)
    
    # Separate features (X) and labels (y)
    X = np.array([item[0] for item in data])
    y = np.array([item[1] for item in data])
    
    # Reshape features to (N, H, W, C)
    # The -1 means infer the number of samples (N)
    X = X.reshape(-1, IMG_SIZE[0], IMG_SIZE[1], channels)
    
    # Normalize features
    X = X.astype('float32') / 255.0
    
    # Convert labels to categorical (one-hot encoding)
    y = to_categorical(y, num_classes=len(classes))
    
    print(f"[INFO] Features shape: {X.shape}")
    print(f"[INFO] Labels shape: {y.shape}")
    
    return X, y

In [ ]:
# -------------------------------------------------------------------
# 3. BALANCED MODEL - OPTIMAL CAPACITY + MODERATE REGULARIZATION
# -------------------------------------------------------------------

def create_balanced_simpsons_classifier(IMG_SIZE, channels, num_classes):
    """
    Balanced model with:
    - Light data augmentation (flip + gentle rotation only)
    - Moderate model complexity (32 -> 64 -> 96 filters)
    - Moderate dropout rates (0.3 conv, 0.5 dense)
    - Batch normalization for stability
    """
    model = Sequential()
    
    # LIGHT DATA AUGMENTATION (only gentle transformations)
    model.add(RandomFlip("horizontal", input_shape=(IMG_SIZE[0], IMG_SIZE[1], channels)))
    model.add(RandomRotation(0.1))  # ±10 degrees (reduced from ±15)
    
    # CONVOLUTIONAL LAYERS - BALANCED COMPLEXITY
    # Block 1: 32 filters (keep original capacity)
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))  # Moderate dropout
    
    # Block 2: 64 filters (keep original capacity)
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))  # Moderate dropout

    # Block 3: 96 filters (slight reduction from 128)
    model.add(Conv2D(96, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(96, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))  # Moderate dropout
    
    # CLASSIFICATION HEAD - BALANCED CAPACITY
    model.add(Flatten())
    model.add(Dense(384, activation='relu'))  # Moderate reduction from 512
    model.add(BatchNormalization())
    model.add(Dropout(0.5))  # Standard dropout for dense layer
    model.add(Dense(num_classes, activation='softmax'))  # Output layer
    
    # Compile with Adam optimizer
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# -------------------------------------------------------------------
# 4. EXECUTION
# -------------------------------------------------------------------

# 4.1 Create the training data
X_train, y_train = preprocess_images_custom(char_path, characters, IMG_SIZE, channels)

# Clear memory
gc.collect()

In [ ]:
# 4.2 Create the balanced model
model = create_balanced_simpsons_classifier(IMG_SIZE, channels, num_classes)

# Display model architecture
print("="*60)
print("BALANCED MODEL ARCHITECTURE")
print("="*60)
model.summary()
print("="*60)

In [ ]:
# 4.3 Setup optimized training configuration
BATCH_SIZE = 64  # Increased from 32 for more stable gradients
EPOCHS = 30  # Max epochs with early stopping

# OPTIMIZED CALLBACKS
callbacks = [
    # Early Stopping: Monitor validation accuracy with more patience
    EarlyStopping(
        monitor='val_accuracy',  # Changed from val_loss
        patience=7,  # Increased from 5 to give more time
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    
    # Model Checkpoint: Save the best model
    ModelCheckpoint(
        'simpsons_classifier_best.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Reduce Learning Rate: More conservative reduction
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,  # Increased from 3
        min_lr=1e-7,
        verbose=1
    )
]

print("="*60)
print("OPTIMIZED TRAINING CONFIGURATION")
print("="*60)
print(f"Batch Size: {BATCH_SIZE} (increased for stability)")
print(f"Max Epochs: {EPOCHS}")
print(f"Validation Split: 20%")
print(f"Early Stopping: Monitor val_accuracy, patience=7")
print(f"Model Checkpoint: Save best val_accuracy")
print(f"Learning Rate: Start at 0.001, reduce on plateau")
print("="*60)

In [ ]:
# 4.4 Train the balanced model
print("\n" + "="*60)
print("STARTING TRAINING - BALANCED APPROACH")
print("="*60)
print("Target: 90%+ validation accuracy with <5% train/val gap")
print("="*60 + "\n")

history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)

In [ ]:
# 4.5 Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
plt.axhline(y=0.90, color='g', linestyle='--', label='90% Target', alpha=0.7)
plt.title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
accuracy_gap = (final_train_acc - final_val_acc) * 100

# Find best validation accuracy
best_val_acc = max(history.history['val_accuracy'])
best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1

print("\n" + "="*60)
print("FINAL TRAINING METRICS")
print("="*60)
print(f"Final Training Accuracy:   {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
print(f"Final Validation Accuracy: {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
print(f"Final Training Loss:       {final_train_loss:.4f}")
print(f"Final Validation Loss:     {final_val_loss:.4f}")
print(f"Accuracy Gap:              {accuracy_gap:.2f}%")
print("-" * 60)
print(f"Best Validation Accuracy:  {best_val_acc:.4f} ({best_val_acc*100:.2f}%) at epoch {best_epoch}")
print("="*60)

# Success check
if final_val_acc >= 0.90:
    print("\n🎉 SUCCESS! Validation accuracy >= 90%")
elif final_val_acc >= 0.85:
    print("\n✅ GOOD! Validation accuracy >= 85%")
elif final_val_acc >= 0.80:
    print("\n👍 IMPROVED! Validation accuracy >= 80%")
else:
    print("\n⚠️  Still below 80% - may need further tuning")

if abs(accuracy_gap) < 5:
    print("✅ Excellent generalization (gap < 5%)")
elif abs(accuracy_gap) < 10:
    print("👍 Good generalization (gap < 10%)")
else:
    print("⚠️  Generalization gap still high")

print("="*60)

In [ ]:
# Clear training data from memory
del X_train
del y_train
gc.collect()

In [ ]:
# 4.6 Prediction Utility
def prepare_image(img_path, IMG_SIZE, channels):
    try:
        if channels == 1:
            img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
        else:
            img = cv.imread(img_path)
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            
        if img is None:
            raise FileNotFoundError(f"Image not found at {img_path}")
            
        resized_img = cv.resize(img, IMG_SIZE)
        reshaped_img = resized_img.reshape(1, IMG_SIZE[0], IMG_SIZE[1], channels) / 255.0
        
        return reshaped_img
    except Exception as e:
        print(f"Error in prepare_image: {e}")
        return None

In [ ]:
# 4.7 Example Prediction and Model Saving
test_img_path = os.path.join(char_path, characters[0], os.listdir(os.path.join(char_path, characters[0]))[0])

prepared_img = prepare_image(test_img_path, IMG_SIZE, channels)

if prepared_img is not None:
    predictions = model.predict(prepared_img, verbose=0)
    predicted_index = np.argmax(predictions[0])
    predicted_class = characters[predicted_index]
    confidence = predictions[0][predicted_index]
    
    print("\n" + "="*60)
    print("SAMPLE PREDICTION TEST")
    print("="*60)
    print(f"Test Image: {os.path.basename(test_img_path)}")
    print(f"Predicted Character: {predicted_class}")
    print(f"Confidence: {confidence*100:.2f}%")
    print("="*60)

    # Save the final model
    model.save('simpsons_classifier.h5')
    print("\n[INFO] ✅ Model saved to 'simpsons_classifier.h5'")
    print("[INFO] ✅ Best model saved to 'simpsons_classifier_best.h5'")
    print("\n" + "="*60)
else:
    print("⚠️  Skipping prediction due to image loading error.")